# Theory 

## Anomaly Detection in Keras 

- Anomaly Detection is unsupervised training tech 
- Analyzes the degree to which incoming data is different than data that you used to train the neural network
- Cybersecurity used anomaly detection to ensure network security 

---
## In Game 
- Already so many company use Anomaly Detection finding aka. GameBot

## **google cloud tech presentation** - in May 10, 2019
- Netmable

from IPython.display import YouTubeVideo
YouTubeVideo('xrZ9BRK5WAg', width=600, height=480)

<img src="https://blog.kakaocdn.net/dn/2EwaL/btq60IMBfzV/2TWCr5YyVhHWLjRrj2P3T1/img.png" align ="left">

- 기존 규칙 기반의 봇 탐지 보다 확실히 좋은.. 성능

---

### LSTM Autoencoder을 통한 Anomaly Detection

{Autoencoder : 자기학습 모델}

- Autoencoder의 경우 보통 이미지의 생성이나 복원에 많이 사용되며 
- 구조를 이어받아 대표적인 딥러닝 생성 모델인 GAN(Generative Adversarial Network)으로 까지 이어짐

- **LSTM Autoencoder 학습 시에는 정상(normal) 신호의 데이터로만 모델을 학습 시킨다**

- encoder와 decoder는 학습이 진행될 수록 정상 신호를 정산 신호 답게 표현 하는 방법을 학습하고
- 최종적으로 재 구성한 결과도 정산 신호와 유사한 분포를 가지는 데이터!!가 된다.

<img src="https://jaehyeongan.github.io/image/lstm-autoencoder-architecture2.png">

- 만약에 비 정상 신호를 입력에 넣게 된다면 정상 분포와 다른 특성의 분포를 나타낼것이기에 높은 reconstruction error가 생길것이다

## Threshold by precision-Recall-Curve
- Autoencoder에서 재구성된 결과 vs input를 비교 하여서 Reconstruction error를 계산하는데
- error val 이 낮다면 정상, 높다면 비정상으로 판단하기로 하였다
- 그러면 이 판단 기준은 어떻게 짜야 할까!?


- 보통 classification 같은 경우는 50%를 기준으로 판단을 하지만
- Reconstruction error 같은 경우는 극단적으로 값이 튀기 어렵기에.. 
- 타당한 threshold 값을 정하는 것이 필요하다

# Precision Recall Val

<img src="https://jaehyeongan.github.io/image/precision-recall-curve.png">

- recall과 precision이 tradoff 관계인 점을 이용해서 한쪽에 치우치는 않는 케이스를 찾기 위한 과정으로 
- precision recall curve를 사용해준다

## Start example 

#### KDD99 DATA SET
- Old dataset 20 years old 
- Still widely used to demonstarate intrusion Detection system 
- Used for international knowledge Discovery and Data Mining Tools Competition

### Tasks 

- build a network intrusion detector **침입 감지**
    - predictive model capable of distinguishing between **"bad"**, **"good"** connection 

> TMI

- This database contains a standard set of data to be audited,
- including a wide variety of intrusions simulated in a military network environment.

##### Note 
- kdd99 does not include column names, caus to old 

In [88]:
import pandas as pd
from tensorflow.keras.utils import get_file

pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

try:
    path = get_file('kddcup.data_10_percent.gz', origin=\
    'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz')
except:
    print('Error downloading')
    raise
    
print(path) 

df = pd.read_csv(path, header=None)

print("Read {} rows.".format(len(df)))

df.dropna(inplace=True,axis=1) 

df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'
]

# display 5 rows
pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 5)
df

/home/flycode77/.keras/datasets/kddcup.data_10_percent.gz
Read 494021 rows.


,duration,protocol_type,...,dst_host_srv_rerror_rate,outcome
0,0,tcp,...,0.0,normal.
1,0,tcp,...,0.0,normal.
...,...,...,...,...,...
494019,0,tcp,...,0.0,normal.
494020,0,tcp,...,0.0,normal.


In [89]:
# "outcome" specifies either "normal," indicating no attack, or the type of attack performed. 

In [90]:
df.groupby('outcome')["outcome"].count()

outcome
back.               2203
buffer_overflow.      30
                    ... 
warezclient.        1020
warezmaster.          20
Name: outcome, Length: 23, dtype: int64

### Preprocessing

- Before MLP learn need some processing 
    - 1. The first function converts numeric columns into Z-Scores. # 정규화
    - 2. The second function replaces categorical values with dummy variables.

In [91]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] 
# for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


In [92]:
# Now encode the feature vector

pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

encode_numeric_zscore(df, 'duration')
encode_text_dummy(df, 'protocol_type')
encode_text_dummy(df, 'service')
encode_text_dummy(df, 'flag')
encode_numeric_zscore(df, 'src_bytes')
encode_numeric_zscore(df, 'dst_bytes')
encode_text_dummy(df, 'land')
encode_numeric_zscore(df, 'wrong_fragment')
encode_numeric_zscore(df, 'urgent')
encode_numeric_zscore(df, 'hot')
encode_numeric_zscore(df, 'num_failed_logins')
encode_text_dummy(df, 'logged_in')
encode_numeric_zscore(df, 'num_compromised')
encode_numeric_zscore(df, 'root_shell')
encode_numeric_zscore(df, 'su_attempted')
encode_numeric_zscore(df, 'num_root')
encode_numeric_zscore(df, 'num_file_creations')
encode_numeric_zscore(df, 'num_shells')
encode_numeric_zscore(df, 'num_access_files')
encode_numeric_zscore(df, 'num_outbound_cmds')
encode_text_dummy(df, 'is_host_login')
encode_text_dummy(df, 'is_guest_login')
encode_numeric_zscore(df, 'count')
encode_numeric_zscore(df, 'srv_count')
encode_numeric_zscore(df, 'serror_rate')
encode_numeric_zscore(df, 'srv_serror_rate')
encode_numeric_zscore(df, 'rerror_rate')
encode_numeric_zscore(df, 'srv_rerror_rate')
encode_numeric_zscore(df, 'same_srv_rate')
encode_numeric_zscore(df, 'diff_srv_rate')
encode_numeric_zscore(df, 'srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_count')
encode_numeric_zscore(df, 'dst_host_srv_count')
encode_numeric_zscore(df, 'dst_host_same_srv_rate')
encode_numeric_zscore(df, 'dst_host_diff_srv_rate')
encode_numeric_zscore(df, 'dst_host_same_src_port_rate')
encode_numeric_zscore(df, 'dst_host_srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_serror_rate')
encode_numeric_zscore(df, 'dst_host_srv_serror_rate')
encode_numeric_zscore(df, 'dst_host_rerror_rate')
encode_numeric_zscore(df, 'dst_host_srv_rerror_rate')

# display 5 rows

df.dropna(inplace=True,axis=1)
df[0:5]

,duration,src_bytes,dst_bytes,...,is_host_login-0,is_guest_login-0,is_guest_login-1
0,-0.067792,-0.002879,0.138664,...,1,1,0
1,-0.067792,-0.002820,-0.011578,...,1,1,0
2,-0.067792,-0.002824,0.014179,...,1,1,0
3,-0.067792,-0.002840,0.014179,...,1,1,0
4,-0.067792,-0.002842,0.035214,...,1,1,0


# TO DO ANOMALY DETECTION 
- Perform anomaly detection, 
- Data need to divide by two grouop ("Normal", "various Attack ")

In [93]:
normal_mask = df['outcome']=='normal.'
attack_mask = df['outcome']!='normal.'

df.drop('outcome',axis=1,inplace=True)

df_normal = df[normal_mask]
df_attack = df[attack_mask]

print(f"Normal count: {len(df_normal)}")
print(f"Attack count: {len(df_attack)}")

Normal count: 97278
Attack count: 396743


In [94]:
# Convert DataFrame into Numpy arrays, keras need numpy
x_normal = df_normal.values
x_attack = df_attack.values

## Training the Autoencoder 

- important to note that we are not using the outcome column as a label to predict
- This anomaly detection is unsupervised
- **no target (y)**
- train an autoencoder on the normal data 
- see how well it can detect that the data not flagged as "normal" represents an anomaly

In [95]:
from sklearn.model_selection import train_test_split

x_normal_train, x_normal_test = train_test_split(
    x_normal, test_size=0.3, random_state=77)

In [96]:
print(f"Normal train count: {len(x_normal_train)}")
print(f"Normal test count: {len(x_normal_test)}")

Normal train count: 68094
Normal test count: 29184


# How to learn?

- The autoencoder will learn to compress the data to a vector of just three numbers
- autoencoder should be able to also decompress with reasonable accuracy

In [97]:
from sklearn import metrics
import numpy as np
import pandas as pd
from IPython.display import display, HTML 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(25, input_dim=x_normal.shape[1], activation='relu'))
model.add(Dense(3, activation='relu')) # size to compress to

model.add(Dense(25, activation='relu'))
model.add(Dense(x_normal.shape[1])) # Multiple output neurons

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_normal_train,x_normal_train,verbose=1,epochs=100)

Epoch 1/100
68094/68094 [==============================] - 2s 35us/sample - loss: 0.4558
Epoch 2/100
68094/68094 [==============================] - 1s 20us/sample - loss: 0.3404
Epoch 3/100
68094/68094 [==============================] - 1s 20us/sample - loss: 0.2653
Epoch 4/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.2524
Epoch 5/100
68094/68094 [==============================] - 1s 20us/sample - loss: 0.1907
Epoch 6/100
68094/68094 [==============================] - 1s 20us/sample - loss: 0.1941
Epoch 7/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1948
Epoch 8/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1834
Epoch 9/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1993
Epoch 10/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1783
Epoch 11/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1800
Epoch 12/100
68094/

68094/68094 [==============================] - 1s 19us/sample - loss: 0.1039
Epoch 93/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1045
Epoch 94/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1017
Epoch 95/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1029
Epoch 96/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1044
Epoch 97/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1008
Epoch 98/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1008
Epoch 99/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.1034
Epoch 100/100
68094/68094 [==============================] - 1s 19us/sample - loss: 0.0937


# Detecting an Anomaly 

- encoding data that represents an attack. This higher error indicates an anomaly.

In [98]:
pred = model.predict(x_normal_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_normal_test))
pred = model.predict(x_normal)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_normal))
pred = model.predict(x_attack)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_attack))
print(f"Out of Sample Normal Score (RMSE): {score1}")
print(f"Insample Normal Score (RMSE): {score2}")
print(f"Attack Underway Score (RMSE): {score3}")

Out of Sample Normal Score (RMSE): 0.3306574651748449
Insample Normal Score (RMSE): 0.32958383415984016
Attack Underway Score (RMSE): 0.5283584940098758
